<a href="https://colab.research.google.com/github/qulacs/quantum-native-dojo/blob/dev/notebooks/4_NISQ_Algorithms" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Qulacs & openfermion & psi4 で VQE

必要なもの

- qulacs
- openfermion
- openfermion-psi4
- psi4
- scipy
- numpy

In [1]:
!pip install qulacs-gpu
!pip install openfermion
!pip install openfermionpsi4

    100% |████████████████████████████████| 133kB 6.3MB/s 
  Running setup.py bdist_wheel for qulacs-gpu ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/9a/e8/85/96c72d5009128fdb28a8b0f5eac7bc04b9a60efe319bc7cf07
Successfully built qulacs-gpu
    100% |████████████████████████████████| 614kB 11.5MB/s 
    100% |████████████████████████████████| 348kB 28.4MB/s 
  Running setup.py bdist_wheel for openfermion ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/c6/24/16/1ee82a50a11993a78c73f1fe208cd83c5c69d0930c1ca1817b
  Running setup.py bdist_wheel for pubchempy ... - done
  Stored in directory: /root/.cache/pip/wheels/10/4d/51/6b843681a9a5aef35f0d0fbce243de46f85080036e16118752
Successfully built openfe

In [3]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local
!conda config --add channels http://conda.anaconda.org/psi4
!python -m conda install -y psi4

--2019-01-20 06:20:02--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.16.19.10, 104.16.18.10, 2606:4700::6810:130a, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.16.19.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M  98.4MB/s    in 5.9s    

2019-01-20 06:20:08 (92.8 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ saved [577996269/577996269]

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-hc7b2577_8 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libgfortran-ng-7.2.0-h9f7466a_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: bzip2-1.0.6-h9a117a8_

In [0]:
import sys, os
import qulacs
sys.path.append('/usr/local/lib/python3.6/site-packages/')

from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from openfermion.hamiltonians import MolecularData
from openfermionpsi4 import run_psi4

from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt



In [13]:
import psi4
psi4.set_memory('500 MB')

h2o = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
""")

psi4.energy('scf/cc-pvdz')

-76.02663273489907

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## ハミルトニアンを作る

In [15]:
basis = "sto-3g"
multiplicity = 1
charge = 0
trotter_step = 1
distance  = 0.977
geometry = [["H", [0,0,0]],["H", [0,0,distance]]]
description  = "tmp"
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_psi4(molecule,run_scf=1,run_fci=1)

n_qubit = molecule.n_qubits
n_electron = molecule.n_electrons

/usr/local/lib/python3.6/dist-packages/openfermionpsi4/_run_psi4.py:226: Warning: No calculation saved. Psi4 segmentation fault possible.
  Warning)


### ハミルトニアンを変換・表示する

In [8]:
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

MissingCalculationError: ignored

In [0]:
print(jw_hamiltonian)

### ハミルトニアンを qulacs ハミルトニアンに変換する

In [0]:
from qulacs import Observable
from qulacs.observable import create_observable_from_openfermion_text

In [0]:
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

## ansatz を構成する

In [0]:
from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import CZ, RY, RZ, merge

depth = n_qubit

In [0]:
def he_ansatz_circuit(n_qubit, depth, theta_list):
    """he_ansatz_circuit
    Retunrs hardware efficient ansatz circuit.

    Args:
        n_qubit (:class:`int`):
            the number of qubit used (equivalent to the number of fermionic modes)
        depth (:class:`int`):
            depth of the circuit.
        theta_list (:class:`numpy.ndarray`):
            rotation angles.
    Returns:
        :class:`qulacs.QuantumCircuit`
    """
    circuit = QuantumCircuit(n_qubit)
    
    for d in range(depth):
        for i in range(n_qubit):
            circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*d]), RZ(i, theta_list[2*i+1+2*n_qubit*d])))
        for i in range(n_qubit//2):
            circuit.add_gate(CZ(2*i, 2*i+1))
        for i in range(n_qubit//2-1):
            circuit.add_gate(CZ(2*i+1, 2*i+2))
    for i in range(n_qubit):
        circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*depth]), RZ(i, theta_list[2*i+1+2*n_qubit*depth])))
    
    return circuit

## Hartree Fock エネルギーのチェック

In [0]:
input_state = QuantumState(n_qubit)
input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))

In [0]:
qulacs_hamiltonian.get_expectation_value(input_state)

In [0]:
molecule.hf_energy

## VQE の cost function

In [0]:
def cost(theta_list):
    input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))
    circuit = he_ansatz_circuit(n_qubit, depth, theta_list)
    circuit.update_quantum_state(input_state)
    return qulacs_hamiltonian.get_expectation_value(input_state)

## 初期パラメータセット

In [0]:
init_theta_list = np.random.random(2*n_qubit*(depth+1))*1e-1
init_theta_list

## 最適化

In [0]:
method = "BFGS"
options = {"disp": True, "maxiter": 50, "gtol": 1e-6}
opt = minimize(cost, init_theta_list, method = method)

In [0]:
opt.x

In [0]:
print(opt.fun, float(molecule.fci_energy))